In [ ]:
import pandas as pd
from pathlib import Path

def rtn_calc(list, end_date="2025-10-31"):
    files_list_series = pd.Series(
        [
            i
            for i in Path("./data").rglob("*")
            if i.suffix.lower() in {".csv", ".xlsx", ".xls"}
        ]
    )
    all_list = []
    for i in list:
        nav_df_path = files_list_series[files_list_series.apply(lambda x: i[1] in x.stem)].values[0]
        nav_df = pd.read_csv(nav_df_path)
        rtn = nav_df[nav_df['日期'] == end_date]['复权净值'].values[0] / nav_df[nav_df['日期'] == i[0]]['复权净值'].values[0] - 1
        all_list.append([i[0],end_date,i[1],i[2],rtn])
    data = pd.DataFrame(all_list,columns=['推荐日期','截止日期','产品代码','产品名称','期间收益率'])
    data['期间收益率'] = data['期间收益率'].apply(lambda x: f"{x:.2%}")
    return data

if __name__ == "__main__":
    recommend_list = [
        ["2025-08-15","SCV067","众壹资产量合兴成一号"],
        ["2025-09-19","SEA420","东海龙王一号"],
        ["2025-09-26","SEM760","草本优益1号"]
    ]
    data = rtn_calc(recommend_list, end_date="2025-10-31")
    print(data)

         推荐日期        截止日期    产品代码        产品名称   期间收益率
0  2025-08-15  2025-10-31  SCV067  众壹资产量合兴成一号  -3.69%
1  2025-09-19  2025-10-31  SEA420      东海龙王一号   2.96%
2  2025-09-26  2025-10-31  SEM760      草本优益1号  -2.76%
